In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
ROOT="/content/drive/MyDrive/Colab_Notebooks/NLP/XLM_Roberta_Language_detection"
os.chdir(ROOT)

In [3]:
os.getcwd()

'/content/drive/MyDrive/Colab_Notebooks/NLP/XLM_Roberta_Language_detection'

In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
#!pip install transformers[sentencepiece]
!pip install transformers==4.28.0
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import pandas as pd
data=pd.read_csv('language_detection.csv')

data.head()

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch


In [7]:
data.shape

(22000, 2)

In [8]:
data.describe()

,Text,language
count,22000,22000
unique,21859,22
top,haec commentatio automatice praeparata res ast...,Estonian
freq,48,1000


In [9]:
label=data['language'].unique().tolist()
labels=sorted(label)

NUM_LABELS= len(labels)

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Text      22000 non-null  object
 1   language  22000 non-null  object
dtypes: object(2)
memory usage: 343.9+ KB


In [11]:
id2label = {idx: labels[idx] for idx in range(len(labels))}
label2id = {v: k for k, v in id2label.items()}
label2id

{'Arabic': 0,
 'Chinese': 1,
 'Dutch': 2,
 'English': 3,
 'Estonian': 4,
 'French': 5,
 'Hindi': 6,
 'Indonesian': 7,
 'Japanese': 8,
 'Korean': 9,
 'Latin': 10,
 'Persian': 11,
 'Portugese': 12,
 'Pushto': 13,
 'Romanian': 14,
 'Russian': 15,
 'Spanish': 16,
 'Swedish': 17,
 'Tamil': 18,
 'Thai': 19,
 'Turkish': 20,
 'Urdu': 21}

In [12]:
def encode_labels(data):
    """Map string labels to integers."""

    data['label']=data.apply(lambda row: label2id[row['language']], axis=1)
    return data

In [13]:
data=encode_labels(data)

In [14]:
data.head()

,Text,language,label
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian,4
1,sebes joseph pereira thomas på eng the jesuit...,Swedish,17
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai,19
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil,18
4,de spons behoort tot het geslacht haliclona en...,Dutch,2


In [15]:
x=list(data['Text'])
y=list(data['label'])
from sklearn.model_selection import train_test_split
train_x,test_x, train_y, test_y=train_test_split(x,y, test_size=0.20,stratify=y, random_state=0)

In [16]:
len(test_x), len(train_x)

(4400, 17600)

In [17]:
from transformers import AutoTokenizer
#from transformers import  BertModel

checkpoint="bert-base-multilingual-cased"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)


train_encoding=tokenizer(train_x, truncation=True, padding=True, return_tensors="pt")
test_encoding=tokenizer(test_x, truncation=True, padding=True, return_tensors="pt")

In [18]:
from transformers import AutoModelForSequenceClassification

model=AutoModelForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=NUM_LABELS, id2label=id2label, label2id=label2id)
model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

## Huggingface login

In [21]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been saved in your

In [20]:
! git config --global credential.helper store

## Convert Encodings into object

In [22]:
import torch

class LangDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = LangDataset(train_encoding, train_y)
val_dataset = LangDataset(test_encoding, test_y)


## Training Arguments

In [ ]:
#pip install --upgrade accelerate

In [23]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir='./results',          # output directory
                                  num_train_epochs=3,              # total number of training epochs
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=8,    # batch size per device during training
                                  per_device_eval_batch_size=16,   # batch size for evaluation
                                  warmup_steps=100,                # number of warmup steps for learning rate scheduler
                                  weight_decay=0.01,               # strength of weight decay
                                  evaluation_strategy="epoch",
                                  logging_dir='./logs',            # directory for storing logs
                                  logging_steps=10,
                                  push_to_hub=True,
                                  hub_model_id='Multilingual_detection')

## Evaluate

In [24]:
from sklearn.metrics import f1_score, accuracy_score, classification_report

def compute_metrics(pred):
    """Custom metric to be used during training."""
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)  # Accuracy
    f1 = f1_score(labels, preds, average="weighted")  # F1-score
    return {
        "Accuracy": acc,
        "F1": f1
        }

## Trainer

In [25]:
trainer = Trainer(
  model,
  args=training_args,
  train_dataset=train_dataset,
  eval_dataset=val_dataset,
  tokenizer=tokenizer,
  compute_metrics=compute_metrics
)


Cloning https://huggingface.co/Manirathinam21/Multilingual_detection into local empty directory.


In [ ]:
# Clearing the CUDA memory cache and free up memory
!pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7393 sha256=3c6c0ddd86380bc0bbfe0b597e473a0a402e27eff320cd020bc1c989f2ae2eb3
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built GPUtil
Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 98% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 37% |  1% |


In [26]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-22-1eee161cda76>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


<ipython-input-22-1eee161cda76>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.002800,0.081287,0.988864,0.989028
2,0.001100,0.064479,0.989773,0.989871
3,0.018700,0.054768,0.988864,0.988909


<ipython-input-22-1eee161cda76>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-22-1eee161cda76>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-22-1eee161cda76>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-22-1eee161cda76>:9: UserWarning: To copy construct from a 

TrainOutput(global_step=6600, training_loss=0.10941060423032578, metrics={'train_runtime': 5721.001, 'train_samples_per_second': 9.229, 'train_steps_per_second': 1.154, 'total_flos': 1.3894758383616e+16, 'train_loss': 0.10941060423032578, 'epoch': 3.0})

In [ ]:
'''
Epoch	Training Loss	 Validation Loss	Accuracy	F1
1     0.002800	      0.081287	      0.988864	0.989028
2	    0.001100	      0.064479	      0.989773	0.989871
3	    0.018700	      0.054768	      0.988864	0.988909
'''

'\nEpoch\tTraining Loss\t Validation Loss\tAccuracy\tF1\n1     0.003000\t      0.083116\t      0.986136\t0.986332\n2\t    0.000900\t      0.069443\t      0.987273\t0.987403\n3\t    0.087900\t      0.067496\t      0.988409\t0.988507\n'

In [27]:
def predict(text):

    # Tokenize the input text and move tensors to the GPU if available
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")

    # Get model output (logits)
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)

    pred_label_idx = probs.argmax()

    # Now map the predicted class index to the actual class label
    # Since pred_label_idx is a tensor containing a single value (the predicted class index),
    # the .item() method is used to extract the value as a scalar
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label

In [28]:
# Test with a an example text in tamil

text = "இதை முயற்சிக்க, கீழே உங்கள் மொழி மற்றும் உள்ளீட்டு கருவியைத் தேர்வுசெய்து, தட்டச்சு செய்யத் தொடங்கவும்."
# "Machine Learning itself is moving towards more and more automated"
predict(text)


(tensor([[2.8965e-05, 1.6313e-05, 1.8858e-05, 2.9802e-05, 2.0865e-05, 1.2371e-05,
          1.1791e-05, 2.3382e-05, 2.6774e-05, 4.0519e-05, 2.1343e-05, 2.3237e-05,
          4.6047e-05, 1.1266e-05, 1.9045e-05, 1.9365e-05, 2.3254e-05, 1.1535e-05,
          9.9954e-01, 1.5674e-05, 1.7697e-05, 2.1531e-05]], device='cuda:0',
        grad_fn=<SoftmaxBackward0>),
 tensor(18, device='cuda:0'),
 'Tamil')

In [ ]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


In [ ]:
model_path = "/content/drive/MyDrive/Colab_Notebooks/NLP/XLM_Roberta_Language_detection"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

Several commits (3) will be pushed upstream.
The progress bars may be unreliable.
